In [83]:
#Funcion para lectura de datos .gpx
import gpxpy
import pandas as pd
import numpy as np
import sqlite3
from geopy.distance import distance

def read_gpx(file: str) -> pd.DataFrame:
    df = None
    points = []
    with open(file) as f:
        gpx = gpxpy.parse(f)

    for segment in gpx.tracks[0].segments:
        for p in segment.points:
            points.append({
                'time':p.time,
                'latitude':p.latitude,
                'longitude':p.longitude,
                'elevation':p.elevation
            })
    df = pd.DataFrame.from_records(points)
    
    return df

In [84]:
#Datos
df1 = read_gpx('recovery.01-Mar-2022-1533.gpx')
df2 = read_gpx('recovery.05-Mar-2022.1025.gpx')
df3 = read_gpx('recovery.25-May-2022-0907.gpx')

In [85]:
#Copias a los datos originales
Data1 = df1.copy()
Data2 = df2.copy()
Data3 = df3.copy()

In [86]:
#Validar si hay datos nulos
Data2.isnull().any()

time         False
latitude     False
longitude    False
elevation    False
dtype: bool

In [87]:
#Validar si hay datos duplicados
Data2.duplicated().any()

False

In [88]:
#Validar si hay filas duplicadas
duplicados = Data2.duplicated()
duplicados

0        False
1        False
2        False
3        False
4        False
         ...  
18588    False
18589    False
18590    False
18591    False
18592    False
Length: 18593, dtype: bool

In [89]:
#mostrar las filas duplicadas
filas_duplicadas = Data2[duplicados]
filas_duplicadas

,time,latitude,longitude,elevation


In [90]:
#Separar Campos de fecha y hora
def separar_campos_fechas(df):
    df["Date_Time"] = pd.to_datetime(df["time"], format="%y-%m-%d").dt.tz_localize(None)
    df['date'] = pd.to_datetime(df['Date_Time'].dt.date)
    df['time'] = df['Date_Time'].dt.time
    df['year'] = df['Date_Time'].dt.year
    df['month'] = df['Date_Time'].dt.month
    df['day'] = df['Date_Time'].dt.day
    df['hour'] = df['Date_Time'].dt.hour
    df['minutes'] = df['Date_Time'].dt.minute
    df['seconds'] = df['Date_Time'].dt.second
    return df

In [91]:
# Calculo de nuevas columnas a los DataFrame
separar_campos_fechas(Data1)
separar_campos_fechas(Data2)
separar_campos_fechas(Data3)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds
0,21:24:46.433000,6.208559,-75.571630,1558.78,2022-05-25 21:24:46.433,2022-05-25,2022,5,25,21,24,46
1,14:05:32.284000,6.268901,-75.593216,1548.51,2022-05-26 14:05:32.284,2022-05-26,2022,5,26,14,5,32
2,14:05:34,6.268870,-75.593087,1551.91,2022-05-26 14:05:34.000,2022-05-26,2022,5,26,14,5,34
3,14:05:36,6.268792,-75.593007,1551.53,2022-05-26 14:05:36.000,2022-05-26,2022,5,26,14,5,36
4,14:05:37,6.268722,-75.592900,1551.37,2022-05-26 14:05:37.000,2022-05-26,2022,5,26,14,5,37
...,...,...,...,...,...,...,...,...,...,...,...,...
653,14:32:21,6.208590,-75.571338,1553.61,2022-05-26 14:32:21.000,2022-05-26,2022,5,26,14,32,21
654,14:32:27,6.208485,-75.571341,1548.82,2022-05-26 14:32:27.000,2022-05-26,2022,5,26,14,32,27
655,14:32:34,6.208457,-75.571435,1549.02,2022-05-26 14:32:34.000,2022-05-26,2022,5,26,14,32,34
656,14:32:41,6.208468,-75.571525,1559.75,2022-05-26 14:32:41.000,2022-05-26,2022,5,26,14,32,41


In [92]:
#Calcular la distancia entre puntos (distancia euclidiana en grados decimales)
def distancia_puntos(df):
    latitud = df['latitude']
    longitud = df['longitude']
    distance = np.sqrt((latitud - latitud.shift())**2 + (longitud - longitud.shift())**2)
    df['distance'] = distance
    return df

In [93]:
distancia_puntos(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34,NaN
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35,0.000020
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43,0.000019
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46,0.000048
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05,2022,3,5,15,25,40,0.000022
18589,15:25:42,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05,2022,3,5,15,25,42,0.000023
18590,15:25:44,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05,2022,3,5,15,25,44,0.000022
18591,15:25:46,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05,2022,3,5,15,25,46,0.000019


In [94]:
#Calcular la velocidad entre puntos (Velocidad en grados decimales / segundos)
def velocidad_puntos(df):
    tiempo = pd.to_datetime(df['Date_Time'])
    time_delta = tiempo.diff().astype('timedelta64[s]')
    df['time_delta'] = time_delta
    speed = df['distance'] / df['time_delta']
    df['speed'] = speed
    return df

In [95]:
velocidad_puntos(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34,NaN,NaN,NaN
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35,0.000020,1.0,0.000020
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43,0.000019,8.0,0.000002
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46,0.000048,3.0,0.000016
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48,0.000024,2.0,0.000012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05,2022,3,5,15,25,40,0.000022,2.0,0.000011
18589,15:25:42,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05,2022,3,5,15,25,42,0.000023,2.0,0.000011
18590,15:25:44,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05,2022,3,5,15,25,44,0.000022,2.0,0.000011
18591,15:25:46,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05,2022,3,5,15,25,46,0.000019,2.0,0.000009


In [96]:
def aceleracion_punto(df):
    aceleracion = df['speed'].diff() / df['time_delta']
    df["acceleration"] = aceleracion
    return df


In [97]:
aceleracion_punto(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed,acceleration
0,00:04:34,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04,2022,3,4,0,4,34,NaN,NaN,NaN,NaN
1,00:04:35,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04,2022,3,4,0,4,35,0.000020,1.0,0.000020,NaN
2,00:04:43,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04,2022,3,4,0,4,43,0.000019,8.0,0.000002,-2.137378e-06
3,00:04:46,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04,2022,3,4,0,4,46,0.000048,3.0,0.000016,4.496680e-06
4,00:04:48,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04,2022,3,4,0,4,48,0.000024,2.0,0.000012,-1.898591e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05,2022,3,5,15,25,40,0.000022,2.0,0.000011,3.264772e-06
18589,15:25:42,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05,2022,3,5,15,25,42,0.000023,2.0,0.000011,1.699839e-07
18590,15:25:44,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05,2022,3,5,15,25,44,0.000022,2.0,0.000011,-2.440891e-07
18591,15:25:46,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05,2022,3,5,15,25,46,0.000019,2.0,0.000009,-6.826574e-07


In [98]:
#Cargar datos a sqlite
def ingestar_sqlite(df, BD = 'Base.db', Tabla = 'Cordenadas'):
    conn = sqlite3.connect(BD)
    df.to_sql(Tabla, conn, if_exists='replace', index = False)
    df_consulta = pd.read_sql(f'SELECT * FROM {Tabla}', conn)
    conn.close()
    return df_consulta

In [99]:
ingestar_sqlite(Data2)

,time,latitude,longitude,elevation,Date_Time,date,year,month,day,hour,minutes,seconds,distance,time_delta,speed,acceleration
0,00:04:34.000000,6.180893,-75.577296,1589.639492,2022-03-04 00:04:34,2022-03-04 00:00:00,2022,3,4,0,4,34,NaN,NaN,NaN,NaN
1,00:04:35.000000,6.180912,-75.577295,1588.165859,2022-03-04 00:04:35,2022-03-04 00:00:00,2022,3,4,0,4,35,0.000020,1.0,0.000020,NaN
2,00:04:43.000000,6.180921,-75.577312,1591.435879,2022-03-04 00:04:43,2022-03-04 00:00:00,2022,3,4,0,4,43,0.000019,8.0,0.000002,-2.137378e-06
3,00:04:46.000000,6.180897,-75.577270,1587.116055,2022-03-04 00:04:46,2022-03-04 00:00:00,2022,3,4,0,4,46,0.000048,3.0,0.000016,4.496680e-06
4,00:04:48.000000,6.180874,-75.577276,1586.772305,2022-03-04 00:04:48,2022-03-04 00:00:00,2022,3,4,0,4,48,0.000024,2.0,0.000012,-1.898591e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18588,15:25:40.000000,6.255059,-75.590578,1481.257603,2022-03-05 15:25:40,2022-03-05 00:00:00,2022,3,5,15,25,40,0.000022,2.0,0.000011,3.264772e-06
18589,15:25:42.000000,6.255073,-75.590597,1480.845982,2022-03-05 15:25:42,2022-03-05 00:00:00,2022,3,5,15,25,42,0.000023,2.0,0.000011,1.699839e-07
18590,15:25:44.000000,6.255086,-75.590614,1480.847569,2022-03-05 15:25:44,2022-03-05 00:00:00,2022,3,5,15,25,44,0.000022,2.0,0.000011,-2.440891e-07
18591,15:25:46.000000,6.255103,-75.590619,1480.186069,2022-03-05 15:25:46,2022-03-05 00:00:00,2022,3,5,15,25,46,0.000019,2.0,0.000009,-6.826574e-07
